# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [8]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd

In [9]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [10]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
document_entity_recognition = tira.pt.transform_documents('ir-lab-sose-2024/ir-nfmj/entity-recognition', pt_dataset)

In [11]:
corpus_with_entities = []

for i in pt_dataset.get_corpus_iter():
    corpus_with_entities += [{'docno': i['docno']}]

corpus_with_entities = document_entity_recognition(pd.DataFrame(corpus_with_entities))

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 60669.18it/s]


In [12]:
def retain_only_entity_type_text(positive_entity_types):
    ret = []
    for _, i in corpus_with_entities.iterrows():
        text = ""
        for j in i["entities"]:
            if j["label"] in positive_entity_types:
                text += " " + j["text"]
        if len(text) > 0:
            ret += [{"docno": i["docno"], "text": text}]     
    return ret        

-----
alle Entitäten

In [13]:
indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, stemmer=None, meta={'docno': 75, 'text': 4096})

index_full_text = pt.IndexFactory.of(indexer.index(pt_dataset.get_corpus_iter()))
bm25 = pt.BatchRetrieve(index_full_text, wmodel="BM25")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  70%|███████   | 88895/126958 [00:10<00:04, 7763.13it/s] 

08:26:41.986 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:14<00:00, 8881.11it/s] 


08:26:47.521 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


In [14]:
indexer = pt.IterDictIndexer("/tmp/index-EVENT", overwrite=True, stemmer=None, meta={'docno': 75, 'text': 4096})

index_event_text = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(set(['EVENT']))))
bm25_event = pt.BatchRetrieve(index_event_text, wmodel="BM25")

08:26:52.024 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (D12-1116) - further warnings are suppressed
08:26:52.192 [ForkJoinPool-3-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


-----
Resultat:
-----
Effektiv: ORG, PERSON, WORK_OF_ART, NORP, PRODUCT, FAC, LOC, MONEY, EVENT, LAW


In [15]:
indexer = pt.IterDictIndexer("/tmp/index-MULTIPLE_TYPES", overwrite=True, stemmer=None, meta={'docno': 75, 'text': 4096})

index_with_entities = pt.IndexFactory.of(indexer.index(retain_only_entity_type_text(set(['ORG', 'PERSON', 'WORK_OF_ART', 'NORP', 'PRODUCT', 'FAC', 'LOC', 'MONEY', 'EVENT', 'LAW']))))
bm25_ents = pt.BatchRetrieve(index_with_entities, wmodel="BM25")

08:26:56.938 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (W11-4616) - further warnings are suppressed
08:27:00.338 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 202 empty documents


-----
Try Improvements
-----

In [16]:
entity_pipe = (bm25*0.99) + (bm25_ents*0.01)

In [17]:
run = entity_pipe(pt_dataset.get_topics())

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


In [18]:
persist_and_normalize_run(run, system_name='bm25-entities', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
